In [1]:
import torch
from torch import nn

class EmbeddingBagClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, num_classes):
        super().__init__()
        # O 'mean' faz a média das palavras da receita
        self.embedding_bag = nn.EmbeddingBag(vocab_size, embedding_dim, mode='mean')
        self.dropout = nn.Dropout(0.5) # Evita que o modelo "decore" em vez de aprender
        self.fc = nn.Linear(embedding_dim, num_classes)

    def forward(self, text, offsets=None):
        embedded = self.embedding_bag(text, offsets)
        embedded = self.dropout(embedded)
        return self.fc(embedded)

In [2]:
dados_treino = [
    ("salada de alface e tomate com cenoura", 1),
    ("suco de laranja com maçã e morango", 0),
    ("espinafre refogado com alho", 1),
    ("salada de frutas com banana e uva", 0),
    ("brócolis ao forno com azeite", 1),
    ("mousse de manga e maracujá", 0)
]

In [3]:
todas_as_palavras = []

# Percorremos cada frase nos nossos dados
for frase, categoria in dados_treino:
    palavras_da_frase = frase.split() # Transforma "suco de uva" em ["suco", "de", "uva"]
    for p in palavras_da_frase:
        todas_as_palavras.append(p)

print(f"Total de palavras encontradas: {len(todas_as_palavras)}")

Total de palavras encontradas: 35


In [5]:
# O set() remove automaticamente o que for repetido
palavras_unicas = set(todas_as_palavras)

vocabulario = {}
id_atual = 0

# Para cada palavra única, damos um número de identificação
for palavra in palavras_unicas:
    vocabulario[palavra] = id_atual
    id_atual = id_atual + 1

vocab_size = len(vocabulario)

print(f"Tamanho do vocabulário: {vocab_size} palavras únicas.")

Tamanho do vocabulário: 24 palavras únicas.


In [6]:
def texto_para_numeros(texto):
    lista_de_numeros = []
    
    palavras = texto.split()
    for p in palavras:
        if p in vocabulario:
            id_da_palavra = vocabulario[p]
            lista_de_numeros.append(id_da_palavra)
    
    # O PyTorch precisa que essa lista vire um "tensor" 
    return torch.tensor(lista_de_numeros, dtype=torch.int64)

In [7]:
frase_exemplo = "salada de frutas"
numeros = texto_para_numeros(frase_exemplo)

print(f"Texto original: {frase_exemplo}")
print(f"Convertido para números: {numeros}") 

Texto original: salada de frutas
Convertido para números: tensor([18, 22, 23])


In [11]:
# Configurações 
embedding_dim = 64
num_classes = 2

modelo = EmbeddingBagClassifier(vocab_size, embedding_dim, num_classes)
otimizador = torch.optim.Adam(modelo.parameters(), lr=0.01)
funcao_perda = nn.CrossEntropyLoss()

# Loop de treinamento simples
for epoca in range(50):
    for texto, label in dados_treino:
        otimizador.zero_grad()
        
        input_tensor = texto_para_numeros(texto).unsqueeze(0) # Prepara o texto
        label_tensor = torch.tensor([label]) # ainda não tinha virado um tensor
        
        output = modelo(input_tensor)
        loss = funcao_perda(output, label_tensor)
        
        loss.backward()
        otimizador.step()

print("Treinamento concluído!")

Treinamento concluído!


In [15]:
def prever(texto):
    modelo.eval()
    with torch.no_grad():
        input_tensor = texto_para_numeros(texto).unsqueeze(0)
        output = modelo(input_tensor)
        predicao = output.argmax(1).item()
        
        categorias = ["Fruta", "Vegetal"]
        return categorias[predicao]

# Teste prático
print(f"Resultado: {prever('suco de morango')}")
print(f"Resultado: {prever('cenoura cozida')}")

Resultado: Fruta
Resultado: Vegetal
